In [1]:
import tensorflow as tf
import numpy as np
import requests
import io

# Step1. 학습데이터 load

In [2]:
"""* Step 1. Inptu tensor 와 Target tensor 준비(훈련데이터)"""

# label 데이터의 각 value 에 해당하는 class name 정보
cifar_label_name=['apple', 'beaver', 'bottle', 'butterfly', 'castle',
'clock', 'couch', 'leopard', 'rose', 'shark']

# 데이터 다운로드 url 경로
data_url = 'https://codepresso-online-platform-public.s3.ap-northeast-2.amazonaws.com/learning-resourse/Tensorflow+2.0+%EB%94%A5%EB%9F%AC%EB%8B%9D+%EC%99%84%EB%B2%BD+%EA%B0%80%EC%9D%B4%EB%93%9C/cifar-10-codepresso.npz'

# requests 라이브러리를 이용해 데이터 다운로드
response = requests.get(data_url)
response.raise_for_status()

# 다운로드 받은 데이터를 읽어 들여 Input tensor 와 Target tensor 준비
with np.load(io.BytesIO(response.content)) as cifar_10_codepresso_data:
  # 학습 이미지 데이터(np.ndarry, shape=(5000, 32, 32, 3))
  train_images = cifar_10_codepresso_data['train_images']
  # 학습 라벨 데이터(np.ndarry, shape=(5000, ))
  train_labels = cifar_10_codepresso_data['train_labels']

  # 테스트 이미지 데이터(np.ndarry, shape=(1000, 32, 32, 3))
  test_images = cifar_10_codepresso_data['test_images']
  # 테스트 라벨 데이터(np.ndarry, shape=(1000, ))
  test_labels = cifar_10_codepresso_data['test_labels']

In [3]:
train_images.shape, train_labels.shape

((5000, 32, 32, 3), (5000,))

In [4]:
test_images.shape, test_labels.shape

((1000, 32, 32, 3), (1000,))

# Step2. 학습데이터의 전처리
* feature  
  * [0-255] => [0-1]
* label  
  * one-hot-encoding

In [5]:
train_images = train_images/255.

test_images = test_images/255.

In [6]:
from tensorflow.keras.utils import to_categorical

In [7]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [8]:
train_labels.shape, test_labels.shape

((5000, 10), (1000, 10))

# Step3. Model 구조 설계  
* input shape (32,32,3)  
* 10개의 클래스로 분류하는 모델 구현

In [9]:
from tensorflow.keras import models, layers

In [10]:
model = models.Sequential()

In [11]:
# 1st Conv2d
#  - filter_cnt : 16 - kernel_size : (3,3) - relu

model.add(layers.Conv2D(filters=16, kernel_size=(3,3),activation='relu',input_shape=(32,32,3)))

# 1st max pooling
#  - pool_size = (2,2)

model.add(layers.MaxPooling2D(pool_size=(2,2)))

# 2nd Conv2d
#  - filter_cnt : 32 - kernel_size : (3,3) - relu

model.add(layers.Conv2D(filters=32, kernel_size=(3,3),activation='relu'))

# 2nd max pooling
#  - pool_size = (2,2))

model.add(layers.MaxPooling2D(pool_size=(2,2)))

# 3rd Conv2d
#  - filter_cnt : 64 - kernel_size : (3,3) - relu

model.add(layers.Conv2D(filters=64, kernel_size=(3,3),activation='relu'))

# Flatten

model.add(layers.Flatten())

# classification module
# hidden layer 추가해도 되고 안해도됨
model.add(layers.Dense(units=10, activation='softmax'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                1

In [13]:
16*(3*3*3) + 16*1

448

# Step4. Model의 학습에 필요한 설정 정보 세팅

In [14]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

# Step5. Model에 학습데이터를 연결하여 학습

In [15]:
model.fit(x=train_images, y=train_labels, epochs=60, batch_size=64 , validation_split=0.2)

Epoch 1/60
63/63 [==============================] - 3s 11ms/step - loss: 1.9223 - accuracy: 0.3090 - val_loss: 2.0842 - val_accuracy: 0.2820
Epoch 2/60
63/63 [==============================] - 0s 4ms/step - loss: 1.6462 - accuracy: 0.4295 - val_loss: 1.7447 - val_accuracy: 0.3890
Epoch 3/60
63/63 [==============================] - 0s 5ms/step - loss: 1.5293 - accuracy: 0.4543 - val_loss: 1.7951 - val_accuracy: 0.4010
Epoch 4/60
63/63 [==============================] - 0s 5ms/step - loss: 1.4355 - accuracy: 0.5008 - val_loss: 1.4720 - val_accuracy: 0.5020
Epoch 5/60
63/63 [==============================] - 0s 5ms/step - loss: 1.3708 - accuracy: 0.5210 - val_loss: 1.4061 - val_accuracy: 0.5120
Epoch 6/60
63/63 [==============================] - 0s 5ms/step - loss: 1.2973 - accuracy: 0.5525 - val_loss: 1.7569 - val_accuracy: 0.4060
Epoch 7/60
63/63 [==============================] - 0s 5ms/step - loss: 1.2451 - accuracy: 0.5680 - val_loss: 1.3360 - val_accuracy: 0.5510
Epoch 8/60
63/63 [=

# Step6. Test 데이터셋을 이용한 모델의 성능 평가

In [16]:
loss, accuracy = model.evaluate(x=test_images, y=test_labels)

32/32 [==============================] - 0s 4ms/step - loss: 1.3432 - accuracy: 0.7060
